dash code modified from [Coding with Adam](https://github.com/Coding-with-Adam/Dash-by-Plotly/tree/master/Good_to_Know/Dash2.0)  
mesh and synapse code modified from Forrest Collman, [Allen Institute](https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb)

In [1]:
from dash import Dash, html, dcc, Output, Input
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [2]:
# read in data tables
clean_complete = pd.read_csv("data/clean_and_complete_soma_ids_v185.csv", names=['cellid'], dtype={'cellid':str})
syn_clean_complete = pd.read_csv('data/syn_clean_complete.csv', index_col=[0])

# define directory location to mesh files
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

In [3]:
# run this cell to download meshes from 65 "clean and complete" curated pyr neurons
# this will put the large mesh files on your local machine in the mesh_dir defined above
# this cell only needs to be run once and will take 10 or more minutes to complete

#clean_complete = pd.read_csv("data/clean_and_complete_soma_ids_v185.csv", names=['cellid'])
#clean_complete_list = clean_complete.cellid.to_list()
#for i in range(len(clean_complete_list)):
#    downloadmesh = mm.mesh(seg_id = clean_complete_list[i], remove_duplicate_vertices=True)


In [4]:
# setting up and loading pyr neuron mesh and synapse data into dictionaries
# this cell will take a minute or more to complete

## create a dictionary of meshes from 65 clean and complete pyr neurons
mesh_dictionary = {}
for i in range(len(clean_complete.cellid.values)):
    mesh_file = os.path.join(mesh_dir, str(clean_complete.cellid.values[i]) +'.h5')
    mesh_dictionary['cellid_' + str(clean_complete.cellid.values[i])] = mm.mesh(filename = mesh_file)

## create dictionaries for pre and post synaptic sites from 65 clean and complete pyr neurons
cell_post_subgraph_dict = {}
cell_pre_subgraph_dict = {}
for i in range(len(clean_complete.cellid.values)):
    cell_post_subgraph_dict['cell_post_subgraph_' + str(clean_complete.cellid.values[i])] = syn_clean_complete.query(f'post_root_id=={clean_complete.cellid.values[i]}')
    cell_pre_subgraph_dict['cell_pre_subgraph_' + str(clean_complete.cellid.values[i])] = syn_clean_complete.query(f'pre_root_id=={clean_complete.cellid.values[i]}')

## create dictionaries for the synapse coordinates from 65 clean and complete pyr neurons
postsyn_xyz_dict = {}
presyn_xyz_dict = {}
for i in range(len(clean_complete.cellid.values)):
    postsyn_xyz_dict['postsyn_xyz_dict_' + str(clean_complete.cellid.values[i])] = cell_post_subgraph_dict.get('cell_post_subgraph_' + str(clean_complete.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values
    presyn_xyz_dict['presyn_xyz_dict_' + str(clean_complete.cellid.values[i])] = cell_pre_subgraph_dict.get('cell_pre_subgraph_' + str(clean_complete.cellid.values[i]))[['ctr_pt_x_nm', 'ctr_pt_y_nm', 'ctr_pt_z_nm']].values


In [5]:
# components

app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN])
load_figure_template("CERULEAN")

mytitle = html.H4(children='')
mygraph = dcc.Graph(figure={})
dropdown = dcc.Dropdown(options=clean_complete.cellid.values,
                        value=str(clean_complete.cellid.values[0]),
                        clearable=False,
                        style={"textAlign":"center"},
                        )
granularity = dcc.Slider(5, 50, 5, value=50, id='granularity')

# layout

app.layout = html.Div(style={"textAlign":"center"},
                      children=[
                          dbc.Row([
                              dbc.Col(width=3),
                              dbc.Col([mytitle])
                          ]),
                          dbc.Row([
                              dbc.Col([
                                  dcc.Markdown('Select granularity'), granularity, 
                                  dcc.Markdown('Select cellid'), dropdown], width=3),
                              dbc.Col(dbc.Card([mygraph]), width=9),
                          ]),
                              ]
                     )

# callback

@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown, 'value'),
    Input(granularity, 'value')
)

def update_graph(cellid, granularity):  # function arguments come from the component property of the Input

    fig = go.Figure(data=[go.Scatter3d(
        x=mesh_dictionary.get('cellid_' + str(cellid)).vertices[::granularity,0],
        y=mesh_dictionary.get('cellid_' + str(cellid)).vertices[::granularity,1],
        z=mesh_dictionary.get('cellid_' + str(cellid)).vertices[::granularity,2],
        mode='markers',
        marker=dict(size=1),
        showlegend=False),
        ])
    
    camera = dict(eye=dict(x=0, y=0.1, z=1.5))
    fig.update_layout(scene_camera=camera, scene_dragmode='orbit')
    
    fig.add_trace(
        go.Scatter3d(
            x=postsyn_xyz_dict.get('postsyn_xyz_dict_' + str(cellid))[:,0],
            y=postsyn_xyz_dict.get('postsyn_xyz_dict_' + str(cellid))[:,1],
            z=postsyn_xyz_dict.get('postsyn_xyz_dict_' + str(cellid))[:,2],
            mode="markers",
            marker=dict(size=2, opacity=0.95),
            showlegend=False)
    )

    fig.add_trace(
        go.Scatter3d(
            x=presyn_xyz_dict.get('presyn_xyz_dict_' + str(cellid))[:,0],
            y=presyn_xyz_dict.get('presyn_xyz_dict_' + str(cellid))[:,1],
            z=presyn_xyz_dict.get('presyn_xyz_dict_' + str(cellid))[:,2],
            mode="markers",
            marker=dict(size=3, opacity=1),
            showlegend=False)
    )

    return fig, 'cellid: '+cellid  # returned objects are assigned to the component property of the Output

# run app
if __name__=='__main__':
    app.run(jupyter_mode="tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>